In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score as score
from sklearn.ensemble import RandomForestClassifier 

In [2]:
train = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/Santander/train.csv")
test = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/Santander/test.csv")

#### I will follow my own approaches and see where I get.

In [3]:
train.tail(1)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
199999,train_199999,0,10.8762,-5.7105,12.1183,8.0328,11.5577,0.3488,5.2839,15.2058,...,8.9842,1.6893,0.1276,0.3766,15.2101,-2.4907,-2.2342,8.1857,12.1284,0.1385


In [4]:
train = train.drop('ID_code', axis =1 )
test = test.drop('ID_code', axis =1)

In [5]:
train.isnull().sum()

target     0
var_0      0
var_1      0
var_2      0
var_3      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 201, dtype: int64

In [6]:
test.isnull().sum()

var_0      0
var_1      0
var_2      0
var_3      0
var_4      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 200, dtype: int64

In [7]:
X_train = train.drop('target', axis =1)
y_train = train['target']

In [8]:
train1 = train.sample(n = 10000, random_state = 1)

In [9]:
X_train1 = train1.drop('target', axis =1)
y_train1 = train1['target']           ### I will also check results for this subset to determine if taking a subset is wise or not

### Determining if taking a subset of 10,000 rows decreases accuracy:

In [10]:
lr = LogisticRegression()

In [11]:
lr.fit(X_train, y_train)

C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
y_train_pred = lr.predict(X_train)

In [13]:
score(y_train_pred, y_train)        ## 91.19%

0.91197

In [14]:
lr1 = LogisticRegression()

lr1.fit(X_train1, y_train1)

C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
score(lr1.predict(X_train1), y_train1)      ## 91.85% accuracy - slightly better - but lets see how it performs on another set

0.9185

In [16]:
train2 = train.sample(n = 100, random_state =2)
X_train2 = train2.drop('target', axis =1)
y_train2 = train2['target']

In [17]:
y_a = lr.predict(X_train2)
y_b = lr1.predict(X_train2)

In [18]:
print(score(y_a, y_train2), score(y_a, y_train2) )         ## barely any difference 

0.86 0.86


### Let's see how this prediction does

In [19]:
X_test = test

In [20]:
y_test_pred = lr1.predict(X_test)

In [21]:
pred1 = pd.DataFrame(y_test_pred)

In [22]:
pred1.value_counts()  ### not bad, lets see how much score this gives me on Kaggle

0    193209
1      6791
Name: count, dtype: int64

In [23]:
sub1 = pred1.to_excel('sub1.xlsx', index = False)

### Gave merely 61% accuracy - not enough

In [24]:
train['target'].value_counts()

target
0    179902
1     20098
Name: count, dtype: int64

#### 20098:179902 = 111.7:1000 is approximately 1120:10000 - which is how many of the target = 1 and 0 values I will have in the new subset

In [25]:
a = train[train['target'] == 0]
b = train[train['target'] == 1]

In [26]:
a1 = train.sample(n = 1120, random_state = 1)
b1 = train.sample(n = 10000, random_state = 1)

In [27]:
train1 = pd.concat([a1, b1])

In [28]:
train1.tail()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
139688,0,9.5185,1.3504,10.3818,3.7889,12.9126,4.6743,5.7923,15.8692,0.9555,...,5.8696,5.8430,1.8926,2.8954,21.7089,0.1885,1.5801,8.4335,15.9431,7.5743
190219,0,10.8011,-2.1339,11.5498,3.6420,9.5415,-10.1326,4.0245,13.2672,-0.2651,...,-1.9895,1.1768,0.0642,3.8913,13.5573,1.6629,6.1016,8.8965,13.2329,-7.8768
58609,0,8.0144,-6.4916,13.0600,5.7138,12.2755,-13.2712,6.3776,12.4741,-7.9000,...,0.7765,9.3877,3.3081,7.2874,17.2969,1.0302,-4.8068,8.1954,11.6955,-20.5529
163592,0,10.4955,4.8713,11.8799,7.3492,8.2514,-4.2326,5.8356,17.1949,-0.6781,...,-0.3068,8.5492,3.1819,-0.6093,21.8977,-0.8353,14.1464,9.3406,16.4803,-12.8834
113353,0,16.1051,7.6987,13.2435,6.1062,9.5795,-4.3834,4.4554,18.0742,-2.8038,...,-1.1394,9.1075,2.3981,6.9050,22.8804,1.0138,-4.6453,9.6388,21.7986,-3.1099


#### This train1 is a great subset for building a model on

In [29]:
X = train1.drop('target', axis =1)
y = train1['target']

In [30]:
xgb = XGBClassifier()

In [31]:
xgb.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

##### I will play around with the parameters of this XGBoost Classifier until I get train accuracy that satisfies me

In [32]:
y1 = xgb.predict(X)

In [33]:
score(y1, y)     ### what?

1.0

In [34]:
y_pred = xgb.predict(X_test)

In [35]:
y2 = pd.DataFrame(y_pred)

In [36]:
y2.value_counts()

0    199082
1       918
Name: count, dtype: int64

In [37]:
sub2 = y2.to_excel('sub2.xlsx', index = False)         

##### this gave such a bad test result that I am not sure I should even consider playing around with its parameters

#### Only logistic regression and Random Forests have given me good results - so I will use those algorithms with parameter tuning to reduce overfitting

# I am going to remove multicollinearity.